<a href="https://colab.research.google.com/github/Nowrland/Projeto_Imersao_Alura/blob/main/ProjetoImersaoAlura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Inicio com a instalação do **Google Generative AI** e **PyPDF2** que é uma biblioteca para manipulação de PDF.

In [1]:
!pip install google-generativeai
!pip install pypdf2
!pip install docx2txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3960 sha256=25bc6abaf0c49c137e9f90e0c37cd905a52e594fa10b3f49973ecce6a07bf553
  Stored in directory: /root/.cache/pip/wheels/22/58/cf/093d0a6c3ecfdfc5f6ddd5524043b88e59a9a199cb02352966
Successfully built docx2txt


Importando as bibliotecas necessarias, e configurando a API.

In [2]:
import google.generativeai as genai
import hashlib
import os
import PyPDF2
import docx2txt
import pathlib
import textwrap
from transformers import pipeline
from pathlib import Path
from google.colab import userdata
from google.colab import files
from IPython.display import display
from IPython.display import Markdown

api_key = userdata.get("SAPI")
genai.configure(api_key=api_key)

Como meu projeto consiste em utilizar um arquivo enviado pelo usuario, vou fazer a requisição do upload do mesmo.

In [3]:
uploaded = files.upload()

Saving Redacao1.docx to Redacao1.docx


Depois de adquirir o arquivou vou armazenar em uma variavel o caminho do arquivo para utilização na extração do texto.

In [7]:
for filename in uploaded.keys():
  uploaded_extension = filename.split('.')[-1]

for filename in uploaded.keys():
    uploadedfile = filename
    file_path = os.path.abspath(uploadedfile)

if uploaded_extension == 'pdf':
    # Extrair texto do PDF
  pdf_file_path = file_path
  with open(pdf_file_path, 'rb') as pdf_file:
      pdf_reader = PyPDF2.PdfReader(pdf_file)
      text = ""
      for page_num in range(len(pdf_reader.pages)):
          page = pdf_reader.pages[page_num]
          text += page.extract_text()

elif uploaded_extension == 'txt' or uploaded_extension == 'docx':
  text = docx2txt.process(file_path)

else:
  print("Extensão não suportada, use pdf, Word(docx) ou texto")

Configurar o modelo e preparar o modelo para utilização.

In [8]:
# Preparar o modelo
generation_config = {
  "temperature": 1,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              safety_settings=safety_settings,
                              )

Com o modelo preparado, agora executaremos as analises e soluções para aprimoração do texto. A partir daqui usarei a generative ai para executar soluções de problemas de texto.

Passo a passo:
1.  Requisitar o arquivo com o texto e o tema da redação;
2.  Ranquear o texto em relação ao tema;
3.  Procurar por conteúdo fora de contexto;
4.  Analisar pontos fracos e fortes do texto;
5.  Sugerir soluções para aprimorar/corrigir pontos fracos;


In [10]:
#Usarei cadeia de dados para executar minhas soluções.
def aprimoramento_texto(text):
  prompt_1 = f"Identifique pontos fracos e incoerencias. Pontue o que precisa ser melhorado no {text}. Faça sugestoes de aprimoramentos do {text} para alcancar a melhorar da escrita da redacao. Organize o resultado da sua analise em topicos."
  sugestoes = model.generate_content(prompt_1)
  return sugestoes.text

tema_texto = input("Digite o Tema: ")

prompt = f"Analise se o conteudo do {text} está compativel com o tema {tema_texto} e dê uma nota de 0 a 10, sendo 0 muito fora do tema e 10 muito condizente com o tema, escreve somente o numero"
ranqueamento_str = model.generate_content(prompt)
ranqueamento_int = int(ranqueamento_str.text)#STR para INT

#Vou eliminar o textos abaixo de 5 porque como é muito fora do tema os aprimoramentos seriam basicamente um texto novo
if ranqueamento_int > 5:
  sugestoes = aprimoramento_texto(text)
  print("\n", sugestoes)
elif ranqueamento_int < 6:
  print("\nO texto está muito ou completamente fora do tema. \n\nFaça um texto mais elaborado e tente novamente.")


Digite o Tema: Rede Sociais

 ## Análise da Redação e Sugestões de Aprimoramento:

**Pontos Fracos e Incoerências:**

* **Generalizações:** Frases como "a população tende a utilizá-la" e "a sensação de que é necessário postar sobre tudo" são generalizações que não se aplicam a todos. É importante qualificar melhor essas afirmações, reconhecendo a diversidade de usos e experiências nas redes sociais.
* **Falta de Especificidade:** A redação menciona "diversos danos" e "distúrbios psíquicos" sem especificar exemplos concretos. Aprofundar a discussão com exemplos mais específicos tornaria a argumentação mais convincente.
* **Transição Abrupta:** A conexão entre a ideia de "sociedade do espetáculo" e os perigos da exposição nas redes sociais não está clara. É preciso aprofundar a relação entre esses conceitos, mostrando como a busca por atenção online se conecta à teoria de Debord.
* **Conclusão Simplória:** A conclusão repete ideias já apresentadas e não oferece soluções concretas além de